<a href="https://colab.research.google.com/github/chaitanyaafle/hedgehog/blob/main/notebooks/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================================
# Project Hedgehog: Uncertainty-Aware LLM Safety Monitor
# Baseline LLM Safety Classifier
# Goal: Establish baseline performance before adding uncertainty quantification
# ============================================================================

"""
NOTEBOOK OVERVIEW:
1. Setup (mount Drive, install packages, check GPU)
2. Download model and data
3. Implement baseline safety classifier
4. Evaluate on SafetyBench
5. Save results to Drive

Runtime: ~30-40 minutes on T4, ~15-20 minutes on A100
"""

'\nNOTEBOOK OVERVIEW:\n1. Setup (mount Drive, install packages, check GPU)\n2. Download model and data\n3. Implement baseline safety classifier\n4. Evaluate on SafetyBench\n5. Save results to Drive\n\nRuntime: ~30-40 minutes on T4, ~15-20 minutes on A100\n'

In [2]:
# ============================================================================
# CELL 1: Environment Setup
# ============================================================================

import sys
import torch
from google.colab import drive

# Mount Google Drive for persistence
drive.mount('/content/drive')

# Create project directory
PROJECT_DIR = '/content/drive/MyDrive/llm_safety_project'
!mkdir -p {PROJECT_DIR}
!mkdir -p {PROJECT_DIR}/data
!mkdir -p {PROJECT_DIR}/results
!mkdir -p {PROJECT_DIR}/models

print(f"✓ Project directory: {PROJECT_DIR}")

# Check GPU
print(f"\n=== GPU Info ===")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime > Change runtime type > T4 GPU")

# Check Python version
print(f"\nPython version: {sys.version}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ Project directory: /content/drive/MyDrive/llm_safety_project

=== GPU Info ===
PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
GPU Memory: 85.17 GB

Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [5]:
# ============================================================================
# CELL 2: Install Dependencies (Clean - No Quantization Needed)
# ============================================================================

# With A100 40GB, we don't need bitsandbytes for 8B models
# This avoids all the CUDA/compatibility issues

!pip install -q --upgrade beartype>=0.16.2
!pip install -q transformer-lens==1.17.0
!pip install -q mapie==0.8.3
!pip install -q accelerate==0.25.0

# Skip sentence-transformers for now (we'll add it on Day 3 when needed)
# This avoids unnecessary dependencies

print("✓ Packages installed (optimized for A100)")

# Verify imports
import mapie
import transformer_lens
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
from datetime import datetime
import torch

print("✓ All imports successful")
print(f"\n📦 Package versions:")
print(f"  transformers: {__import__('transformers').__version__}")
# print(f"  transformer-lens: {transformer_lens.__version__}") # Removed as transformer_lens does not have __version__
print(f"  mapie: {mapie.__version__}")
print(f"  torch: {torch.__version__}")

print(f"\n🎮 GPU Info:")
print(f"  CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  CUDA version: {torch.version.cuda}")
    print(f"  GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"\n  ✅ With {torch.cuda.get_device_properties(0).total_memory / 1e9:.0f}GB, you don't need quantization!")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformer-lens 1.17.0 requires beartype<0.15.0,>=0.14.1, but you have beartype 0.22.6 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
plum-dispatch 2.6.0 requires beartype>=0.16.2, but you have beartype 0.14.1 which is incompatible.
✓ Packages installed (optimized for A100)
✓ All imports successful

📦 Package versions:
  transformers: 4.57.3
  mapie: 0.8.3
  torch: 2.9.0+cu126

🎮 GPU Info:
  CUDA available: True
  CUDA version: 12.6
  GPU: NVIDIA A100-SXM4-80GB
  Memory: 85.17 GB

  ✅ With 85GB, you don't need quantization!


In [4]:
import transformer_lens

In [ ]:
transformer_lens.